# Lesson 1: Advanced RAG Pipeline

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv, find_dotenv

In [3]:
import numpy as np

In [4]:
from trulens_eval import (
    Feedback,
    TruLlama,
    OpenAI
)

from trulens_eval.feedback import Groundedness

In [5]:
import nest_asyncio

In [6]:
import utils

import os
import openai
openai.api_key = os.environ["OPENAI_API_KEY"] # utils.get_openai_api_key()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [7]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [8]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

41 

<class 'llama_index.schema.Document'>
Doc ID: 5af25021-1595-4402-9a4e-f33cfaf00ba5
Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng
How to  Build Your Career in AIA Simple Guide


## Basic RAG pipeline

In [9]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [10]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)
index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [11]:
query_engine = index.as_query_engine()

In [12]:
response = query_engine.query(
    "What are steps to take when finding projects to build your experience?"
)
print(str(response))

When finding projects to build your experience, there are several steps you can take. First, you can join existing projects by asking to join someone else's project if they have an idea. Additionally, you can continue reading, taking courses, and talking to domain experts to come up with new ideas. It is also helpful to focus on an application area that has not yet been explored with machine learning. If your company or school has a particular application in mind, you can explore the possibilities for machine learning in that area. Finally, you can develop a side hustle or personal project that may not initially be part of your job but can help you gain experience and strengthen your skills.


## Evaluation setup using TruLens

In [13]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

What are the keys to building a career in AI?
How can teamwork contribute to success in AI?
What is the importance of networking in AI?
What are some good habits to develop for a successful career?
How can altruism be beneficial in building a career?
What is imposter syndrome and how does it relate to AI?
Who are some accomplished individuals who have experienced imposter syndrome?
What is the first step to becoming good at AI?
What are some common challenges in AI?
Is it normal to find parts of AI challenging?


In [14]:
# You can try your own question:
new_question = "What is the right AI job for me?"
eval_questions.append(new_question)

In [15]:
print(eval_questions)

['What are the keys to building a career in AI?', 'How can teamwork contribute to success in AI?', 'What is the importance of networking in AI?', 'What are some good habits to develop for a successful career?', 'How can altruism be beneficial in building a career?', 'What is imposter syndrome and how does it relate to AI?', 'Who are some accomplished individuals who have experienced imposter syndrome?', 'What is the first step to becoming good at AI?', 'What are some common challenges in AI?', 'Is it normal to find parts of AI challenging?', 'What is the right AI job for me?']


In [16]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


For the classroom, we've written some of the code in helper functions inside a utils.py file.  
- You can view the utils.py file in the file directory by clicking on the "Jupyter" logo at the top of the notebook.
- In later lessons, you'll get to work directly with the code that's currently wrapped inside these helper functions, to give you more options to customize your RAG pipeline.

In [17]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Direct Query Engine")

In [18]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

In [19]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [20]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Answer Relevance_calls,Context Relevance_calls,Groundedness,Groundedness_calls,latency,total_tokens,total_cost
0,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_96c2ce0f3f95f4c799bc53ba5558b8c9,"""What are the keys to building a career in AI?""","""The keys to building a career in AI are learn...",-,"{""record_id"": ""record_hash_96c2ce0f3f95f4c799b...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-24T20:43:42.775628"", ""...",2023-12-24T20:43:49.907947,1.0,0.9,[{'args': {'prompt': 'What are the keys to bui...,[{'args': {'prompt': 'What are the keys to bui...,NaN,NaN,7,2254,0.003500
1,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_54a7d430daae600917a753c54473cb1f,"""How can teamwork contribute to success in AI?""","""Collaborating and working in teams is crucial...",-,"{""record_id"": ""record_hash_54a7d430daae600917a...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-24T20:43:49.996415"", ""...",2023-12-24T20:43:53.579766,1.0,0.0,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'prompt': 'How can teamwork contrib...,1.00,[{'args': {'source': 'Hopefully the previous c...,3,1720,0.002628
2,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_c5bae422ace156892c1bb9dba3a67ed5,"""What is the importance of networking in AI?""","""Networking is important in AI because it allo...",-,"{""record_id"": ""record_hash_c5bae422ace156892c1...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-24T20:43:53.663116"", ""...",2023-12-24T20:43:56.576355,1.0,0.0,[{'args': {'prompt': 'What is the importance o...,[{'args': {'prompt': 'What is the importance o...,0.25,[{'args': {'source': 'Hopefully the previous c...,2,1704,0.002596
3,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_68c7226ca0e965e531eab418f9432ab4,"""What are some good habits to develop for a su...","""Developing good habits is crucial for a succe...",-,"{""record_id"": ""record_hash_68c7226ca0e965e531e...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-24T20:43:56.669833"", ""...",2023-12-24T20:43:59.552546,1.0,0.5,[{'args': {'prompt': 'What are some good habit...,[{'args': {'prompt': 'What are some good habit...,1.00,[{'args': {'source': 'Hopefully the previous c...,2,1678,0.002559
4,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_8f342fe9f5581ae9d8f70be8e79afe51,"""How can altruism be beneficial in building a ...","""Altruism can be beneficial in building a care...",-,"{""record_id"": ""record_hash_8f342fe9f5581ae9d8f...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-24T20:43:59.636593"", ""...",2023-12-24T20:44:02.305371,1.0,0.0,[{'args': {'prompt': 'How can altruism be bene...,[{'args': {'prompt': 'How can altruism be bene...,0.75,[{'args': {'source': 'Hopefully the previous c...,2,1668,0.002539


In [21]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://10.0.0.168:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

## Advanced RAG pipeline

### 1. Sentence Window retrieval

In [22]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [23]:
from utils import build_sentence_window_index

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

In [24]:
from utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

In [25]:
window_response = sentence_window_engine.query(
    "how do I get started on a personal project in AI?"
)
print(str(window_response))

To get started on a personal project in AI, it is important to first identify and scope the project. Consider your career goals and choose a project that complements them. Ensure that the project is responsible, ethical, and beneficial to people. As you progress in your career, aim for projects that grow in scope, complexity, and impact over time. Building a portfolio of projects that shows skill progression can also be helpful. Additionally, there are resources available in the book that provide guidance on starting your AI job search and finding the right AI job for you.


In [26]:
tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine,
    app_id = "Sentence Window Query Engine"
)

In [27]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(str(response))

What are the keys to building a career in AI?
The keys to building a career in AI are learning foundational technical skills, working on projects, and finding a job, all of which is supported by being part of a community.
How can teamwork contribute to success in AI?
Teamwork can contribute to success in AI by allowing individuals to leverage the expertise and insights of their colleagues. When working on larger AI projects, the ability to collaborate effectively with teammates becomes crucial. By working together, team members can share their deep technical insights, make informed decisions about technical architecture or data collection, and ultimately improve the project. Additionally, being surrounded by colleagues who are dedicated, hardworking, and continuously learning can inspire individuals to do the same, leading to greater success in AI endeavors.
What is the importance of networking in AI?
Networking is important in AI because it allows individuals to connect with others wh

In [28]:
tru.get_leaderboard(app_ids=[])

,Groundedness,Answer Relevance,Context Relevance,latency,total_cost
app_id,,,,,
Sentence Window Query Engine,1.0,0.983333,0.166667,2.0,0.000844


In [29]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://10.0.0.168:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### 2. Auto-merging retrieval

In [30]:
from utils import build_automerging_index

automerging_index = build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index"
)

In [31]:
from utils import get_automerging_query_engine

automerging_query_engine = get_automerging_query_engine(
    automerging_index,
)

In [32]:
auto_merging_response = automerging_query_engine.query(
    "How do I build a portfolio of AI projects?"
)
print(str(auto_merging_response))

> Merging 1 nodes into parent node.
> Parent node id: 67b39a93-e052-4378-8662-5ad6b4499c57.
> Parent node text: PAGE 21Building a Portfolio of 
Projects that Shows 
Skill Progression CHAPTER 6
PROJECTS

> Merging 1 nodes into parent node.
> Parent node id: 4fd9c0eb-2bbd-43e3-9010-e2bea0b3d94c.
> Parent node text: PAGE 21Building a Portfolio of 
Projects that Shows 
Skill Progression CHAPTER 6
PROJECTS

To build a portfolio of AI projects, it is important to start with simple undertakings and gradually progress to more complex ones. This progression will demonstrate your growth and development over time. Additionally, effective communication is crucial. You should be able to explain your thought process and the value of your work to others. This will help others see the potential in your projects and trust you with resources for larger endeavors.


In [33]:
tru.reset_database()

tru_recorder_automerging = get_prebuilt_trulens_recorder(automerging_query_engine,
                                                         app_id="Automerging Query Engine")

In [34]:
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)
        print(question)
        print(response)

> Merging 2 nodes into parent node.
> Parent node id: 35af8197-b24d-47f9-ae68-0197fe3e1e40.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...

> Merging 1 nodes into parent node.
> Parent node id: 77692cbd-670f-479a-b9de-c4647952e28d.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...

What are the keys to building a career in AI?
The keys to building a career in AI include learning foundational technical skills, working on projects, finding a job, and being part of a community. Additionally, collaborating with others and being able to influence and be influenced by others is critical for success in AI.
How can teamwork contribute to success in AI?
Teamwork can contribute to success in AI by allowing individuals to collaborate with others, influence their team members, and be influenced by them. This is important because when tackling large projec

In [35]:
tru.get_leaderboard(app_ids=[])

,Answer Relevance,latency,total_cost
app_id,,,
Automerging Query Engine,0.95,1.818182,0.000887


In [36]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://10.0.0.168:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>